In [3]:
# -*- coding: utf-8 -*-

# 이 셀은 노트북에서 가장 먼저 실행하는 공통 준비 셀입니다.
# 경로, 클래스, 시드, 기본 상수를 한 곳에서 관리하기 위해 분리했습니다.

from pathlib import Path
import json
import random
import os

import numpy as np

# 1) 데이터 루트 경로를 설정합니다.
#    AI Stages 환경에서 사용자가 쓰던 경로 패턴을 그대로 맞췄습니다.
FULL_DATA_ROOT = Path("/data/ephemeral/home/model/dataset")

# 2) 원본 train.json 경로를 지정합니다.
#    train/val split을 새로 만들기 위해 필요합니다.
TRAIN_JSON = FULL_DATA_ROOT / "train.json"

# 3) test.json 경로를 지정합니다.
#    추론 시 샘플 제출 파일을 만들 때 필요합니다.
TEST_JSON = FULL_DATA_ROOT / "test.json"

# 4) sample_submission 폴더 경로를 지정합니다.
SAMPLE_SUB_DIR = Path("/data/ephemeral/home/model/sample_submission")

# 5) 실험 결과가 저장될 work_dirs 루트를 지정합니다.
WORK_DIR_ROOT = Path("/data/ephemeral/home/model/work_dirs_single")

# 6) 이번 실험의 모델 이름(폴더명)입니다.
#    학습/추론 셀에서 반드시 동일하게 사용해야 체크포인트를 찾습니다.
EXP_NAME = "sparse_rcnn_r50_1024_stage_aug14"

# 7) 입력 이미지 크기를 통일해서 실험 변수를 줄입니다.
#    사용자의 기존 1024 전략을 유지합니다.
IMAGE_SIZE = (1024, 1024)

# 8) 클래스 정의를 대회 포맷에 맞춰 고정합니다.
CLASSES = (
    "General trash",
    "Paper",
    "Paper pack",
    "Metal",
    "Glass",
    "Plastic",
    "Styrofoam",
    "Plastic bag",
    "Battery",
    "Clothing",
)

# 9) 재현성을 위해 시드를 고정합니다.
SEED = 42

def seed_everything(seed: int = 42):
    # 파이썬 기본 랜덤 시드 고정
    random.seed(seed)
    # 넘파이 랜덤 시드 고정
    np.random.seed(seed)
    # 해시 기반 연산 시드 고정
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(SEED)

# 10) 경로 존재 여부를 사전에 확인해 불필요한 런타임 오류를 줄입니다.
assert TRAIN_JSON.exists(), f"train.json 없음: {TRAIN_JSON}"
assert TEST_JSON.exists(), f"test.json 없음: {TEST_JSON}"
assert SAMPLE_SUB_DIR.exists(), f"sample_submission 폴더 없음: {SAMPLE_SUB_DIR}"

print("준비 완료")
print("EXP_NAME:", EXP_NAME)
print("IMAGE_SIZE:", IMAGE_SIZE)


준비 완료
EXP_NAME: sparse_rcnn_r50_1024_stage_aug14
IMAGE_SIZE: (1024, 1024)


In [4]:
# -*- coding: utf-8 -*-

# 이 셀은 학습이 끝난 후
# work_dir에서 체크포인트를 찾아 test.json 전체에 대해 추론하고,
# sample_submission 템플릿의 순서를 그대로 유지해 제출 csv를 생성합니다.

from pathlib import Path
from typing import Optional, Dict
import json
import pandas as pd

from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules

# 1) 샘플 csv를 선택하는 함수입니다.
def pick_sample_csv(sample_dir: Path) -> Path:
    csvs = sorted(sample_dir.glob("*.csv"))
    if not csvs:
        raise FileNotFoundError(f"샘플 제출 csv 없음: {sample_dir}")
    return csvs[0]

# 2) 체크포인트를 찾는 함수입니다.
def find_checkpoint(work_dir: Path) -> Optional[Path]:
    last_txt = work_dir / "last_checkpoint"
    if last_txt.exists():
        p = last_txt.read_text().strip()
        if p:
            ck = Path(p)
            if not ck.is_absolute():
                ck = work_dir / ck
            if ck.exists():
                return ck

    ckpts = sorted(work_dir.glob("*.pth"))
    if ckpts:
        return ckpts[-1]
    return None

# 3) test.json에서 image_id와 file_name 매핑을 만듭니다.
def parse_test_images(test_json: Path) -> Dict[int, str]:
    with open(test_json, "r") as f:
        data = json.load(f)
    return {img["id"]: img["file_name"] for img in data["images"]}

# 4) COCO bbox 결과 json을 PredictionString dict로 변환합니다.
#    점수 임계값을 너무 낮게 두면 제출 파일이 과도하게 커질 수 있습니다.
#    이전 제출 실패/용량 문제를 고려해 0.15를 기본으로 올려둡니다.
def coco_bbox_json_to_prediction_dict(
    bbox_json: Path,
    test_json: Path,
    score_thr: float = 0.15
) -> Dict[str, str]:

    id_to_fname = parse_test_images(test_json)

    with open(bbox_json, "r") as f:
        dets = json.load(f)

    per_image = {}
    for d in dets:
        score = float(d.get("score", 0.0))
        if score < score_thr:
            continue

        img_id = int(d["image_id"])
        cat_id = int(d["category_id"])

        x, y, w, h = d["bbox"]
        x1, y1, x2, y2 = x, y, x + w, y + h

        per_image.setdefault(img_id, []).append((cat_id, score, x1, y1, x2, y2))

    pred_dict = {}
    for img_id, fname in id_to_fname.items():
        preds = sorted(per_image.get(img_id, []), key=lambda x: x[1], reverse=True)

        tokens = []
        for cat, sc, x1, y1, x2, y2 in preds:
            tokens.extend([
                str(cat),
                f"{sc:.6f}",
                f"{x1:.1f}",
                f"{y1:.1f}",
                f"{x2:.1f}",
                f"{y2:.1f}",
            ])

        pred_dict[fname] = " ".join(tokens)

    return pred_dict

# 5) 샘플 템플릿 순서를 고정해 저장합니다.
def save_submission_with_sample(
    pred_dict: Dict[str, str],
    sample_csv_path: Path,
    out_csv_path: Path
) -> None:

    sample = pd.read_csv(sample_csv_path)

    # 샘플 컬럼 이름을 기준으로 안전 체크
    if "image_id" not in sample.columns or "PredictionString" not in sample.columns:
        raise ValueError(f"샘플 컬럼이 예상과 다름: {list(sample.columns)}")

    # 샘플의 image_id 값은 파일명(file_name)과 동일한 형식이어야 합니다.
    # pred_dict는 file_name 키로 구성되어 있으므로 그대로 매핑합니다.
    sample["PredictionString"] = sample["image_id"].map(lambda x: pred_dict.get(x, ""))

    out_csv_path.parent.mkdir(parents=True, exist_ok=True)
    sample.to_csv(out_csv_path, index=False)

    print("Saved submission:", out_csv_path)

# 6) 경로/실험 폴더를 지정합니다.
MMD_ROOT = Path("/data/ephemeral/home/model/baseline/mmdetection")
WORK_DIR_ROOT = Path("/data/ephemeral/home/model/work_dirs_single")
EXP_WORK_DIR = WORK_DIR_ROOT / EXP_NAME

assert EXP_WORK_DIR.exists(), f"실험 폴더 없음: {EXP_WORK_DIR}"

# 7) 체크포인트를 찾습니다.
ckpt = find_checkpoint(EXP_WORK_DIR)
if ckpt is None:
    raise FileNotFoundError(f"checkpoint 없음: {EXP_WORK_DIR}")

print("checkpoint:", ckpt)

# 8) base cfg를 다시 로드합니다.
#    학습과 동일 계열의 config를 쓰는 것이 안정적입니다.
base_cfg_rel = "configs/sparse_rcnn/sparse-rcnn_r50_fpn_300-proposals_crop-ms-480-800-3x_coco.py"
base_cfg_path = MMD_ROOT / base_cfg_rel
assert base_cfg_path.exists(), f"base cfg 없음: {base_cfg_path}"

register_all_modules(init_default_scope=True)

test_cfg = Config.fromfile(str(base_cfg_path))
test_cfg.default_scope = "mmdet"

# 9) data_root 설정
test_cfg.data_root = str(FULL_DATA_ROOT)

# 10) test_dataloader 구성
if "test_dataloader" not in test_cfg:
    test_cfg.test_dataloader = test_cfg.val_dataloader

loader = test_cfg.test_dataloader
ds_cfg = loader.get("dataset", loader)

ds_cfg.metainfo = dict(classes=CLASSES)
ds_cfg.data_root = str(FULL_DATA_ROOT)
ds_cfg.ann_file = str(TEST_JSON)
ds_cfg.data_prefix = dict(img="")

# 11) test pipeline은 단순/안정적으로 고정합니다.
ds_cfg.pipeline = [
    dict(type="LoadImageFromFile"),
    dict(type="Resize", scale=IMAGE_SIZE, keep_ratio=True),
    dict(type="PackDetInputs"),
]

# 12) num_classes 수정
def set_num_classes(node, num_classes: int):
    if isinstance(node, dict):
        if "num_classes" in node:
            node["num_classes"] = num_classes
        for v in node.values():
            set_num_classes(v, num_classes)
    elif isinstance(node, list):
        for v in node:
            set_num_classes(v, num_classes)

set_num_classes(test_cfg.model, len(CLASSES))

# 13) 배치/워커
test_cfg.test_dataloader.batch_size = 1
test_cfg.test_dataloader.num_workers = 2

# 14) evaluator 설정: COCO bbox json만 생성
out_prefix = EXP_WORK_DIR / f"{EXP_NAME}_test"
test_cfg.test_evaluator = dict(
    type="CocoMetric",
    ann_file=str(TEST_JSON),
    metric="bbox",
    format_only=True,
    outfile_prefix=str(out_prefix),
)

# 15) 체크포인트 로드
test_cfg.load_from = str(ckpt)

# 16) 학습 관련 설정 제거
test_cfg.train_dataloader = None
test_cfg.train_cfg = None
test_cfg.optim_wrapper = None
if hasattr(test_cfg, "param_scheduler"):
    test_cfg.param_scheduler = None
if hasattr(test_cfg, "val_dataloader"):
    test_cfg.val_dataloader = None
if hasattr(test_cfg, "val_cfg"):
    test_cfg.val_cfg = None
if hasattr(test_cfg, "val_evaluator"):
    test_cfg.val_evaluator = None

# 17) work_dir 고정
test_cfg.work_dir = str(EXP_WORK_DIR)

# 18) Runner test 실행
runner = Runner.from_cfg(test_cfg)
runner.test()

# 19) 생성된 bbox json 경로 찾기
bbox_json = EXP_WORK_DIR / f"{EXP_NAME}_test.bbox.json"
if not bbox_json.exists():
    cand = sorted(EXP_WORK_DIR.glob("*_test.bbox.json"))
    if cand:
        bbox_json = cand[-1]

print("bbox json:", bbox_json)
if not bbox_json.exists():
    raise FileNotFoundError("bbox json 생성 실패")

# 20) 제출 csv 생성
sample_csv = pick_sample_csv(SAMPLE_SUB_DIR)
print("Using sample csv:", sample_csv)

pred_dict = coco_bbox_json_to_prediction_dict(
    bbox_json=bbox_json,
    test_json=TEST_JSON,
    score_thr=0.15  # 파일 용량/평가 실패 방지 목적
)

out_csv = EXP_WORK_DIR / f"{EXP_NAME}_submission_full.csv"
save_submission_with_sample(pred_dict, sample_csv, out_csv)

print("추론 완료")
print("제출 파일:", out_csv)


checkpoint: /data/ephemeral/home/model/work_dirs_single/sparse_rcnn_r50_1024_stage_aug14/epoch_14.pth
12/08 14:17:26 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1608637542
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: n/a
    PyTorch: 2.1.2+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gen

/bin/sh: 1: gcc: not found


12/08 14:17:27 - mmengine - INFO - Config:
auto_scale_lr = dict(base_batch_size=16, enable=False)
backend_args = None
data_root = '/data/ephemeral/home/model/dataset'
dataset_type = 'CocoDataset'
default_hooks = dict(
    checkpoint=dict(interval=1, type='CheckpointHook'),
    logger=dict(interval=50, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='DetVisualizationHook'))
default_scope = 'mmdet'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
load_from = '/data/ephemeral/home/model/work_dirs_single/sparse_rcnn_r50_1024_stage_aug14/epoch_14.pth'
log_level = 'INFO'
log_processor = dict(by_epoch=True, type='LogProcessor', window_size=50)
max_epochs = 36
model = dict(
    backbone=dict(
        depth=50,
        frozen_stages=1,
        init_cfg=dict(checkpo